# 🧹 F1 Bronze Cleanup & Reset Utility (Databricks)

This notebook **safely drops and deletes** all F1 bronze Delta tables and their underlying data paths so you can re-run your extraction from a clean state.

It is intended for use in **Databricks** with a Unity Catalog (e.g. `workspace.f1_bronze.session_results`) or legacy metastore (no catalog).

## What this notebook does

For each configured table name (e.g. `events`, `laps`, `session_results`, etc.) it will:

1. Build the **fully-qualified table name** (e.g. `workspace.f1_bronze.session_results`).
2. Check whether the table exists.
3. If it exists:
   - Run `DESCRIBE DETAIL` to find the underlying Delta path.
   - `DROP TABLE` from the metastore.
   - Recursively delete the **data path** using `dbutils.fs.rm(path, recurse=True)`.
4. Log everything so you can see exactly what was done.

> ⚠️ **Warning:** This will permanently delete the underlying Delta data files for the selected tables.


In [0]:
# CONFIGURATION
# ------------------------------------------------------------------
# Adjust these to match your environment.

# If you're using Unity Catalog, set your catalog name here, e.g. "workspace"
# If you're NOT using Unity Catalog, set this to None.
CATALOG = "workspace"  # or None

# Schema / database that holds your F1 bronze tables
SCHEMA = "f1_bronze"

# Bronze tables you want to clean up.
# The script will try each of these in turn.
BRONZE_TABLES = [
    "events",
    "laps",
    "session_results",
    "weather",
    "track_status",
    "race_control_messages",
    "telemetry_raw",
    "car_position",
]

print("✅ Config loaded")
print(f"  CATALOG = {CATALOG!r}")
print(f"  SCHEMA  = {SCHEMA!r}")
print(f"  TABLES  = {BRONZE_TABLES}")

In [0]:
from typing import Optional

def fq_table_name(table: str) -> str:
    """Build a fully qualified table name from CATALOG, SCHEMA, and table."""
    if CATALOG and len(CATALOG.strip()) > 0:
        return f"{CATALOG}.{SCHEMA}.{table}"
    else:
        return f"{SCHEMA}.{table}"


def table_exists(fq_name: str) -> bool:
    """Return True if the table exists in the current Spark catalog."""
    try:
        return spark.catalog.tableExists(fq_name)
    except Exception:
        # Some Spark versions require (db, table) instead of fq string; fall back
        try:
            parts = fq_name.split(".")
            if len(parts) == 3:
                return spark.catalog.tableExists(f"{parts[0]}.{parts[1]}", parts[2])
            elif len(parts) == 2:
                return spark.catalog.tableExists(parts[0], parts[1])
        except Exception:
            return False
    return False


def get_table_path(fq_name: str) -> Optional[str]:
    """Return the underlying Delta path for a table via DESCRIBE DETAIL, or None."""
    try:
        detail_df = spark.sql(f"DESCRIBE DETAIL {fq_name}")
        detail = detail_df.collect()[0].asDict()
        return detail.get("location") or detail.get("path")
    except Exception as e:
        print(f"   ⚠️ Could not DESCRIBE DETAIL {fq_name}: {e}")
        return None


def drop_table(fq_name: str):
    """Drop the table from the metastore if it exists."""
    try:
        spark.sql(f"DROP TABLE IF EXISTS {fq_name}")
        print(f"   ✅ Dropped table {fq_name}")
    except Exception as e:
        print(f"   ⚠️ Error dropping table {fq_name}: {e}")


def delete_path(path: str):
    """Delete a DBFS / cloud filesystem path using dbutils.fs.rm."""
    try:
        if not path:
            return
        print(f"   🗑️ Deleting path: {path}")
        dbutils.fs.rm(path, recurse=True)
        print(f"   ✅ Deleted path: {path}")
    except Exception as e:
        print(f"   ⚠️ Error deleting path {path}: {e}")

In [0]:
# MAIN CLEANUP
# ------------------------------------------------------------------
print("\n🧹 Starting F1 bronze cleanup...\n")

for tbl in BRONZE_TABLES:
    fq_name = fq_table_name(tbl)
    print(f"------------------------------------------------------------")
    print(f"🔎 Checking table: {fq_name}")
    
    if not table_exists(fq_name):
        print(f"   ℹ️ Table does not exist, skipping.")
        continue

    # Try to grab the underlying data path before dropping
    path = get_table_path(fq_name)
    if path:
        print(f"   📁 Data path: {path}")
    else:
        print(f"   ⚠️ No path found via DESCRIBE DETAIL (may be a view or non-Delta table).")

    # Drop the table
    drop_table(fq_name)

    # If we found a location, try to delete it
    if path:
        delete_path(path)

print("\n✅ Cleanup complete. You can now re-run your F1 bronze extraction notebook from a clean state.\n")